In [1]:
# import modules
import numpy as np
import pandas as pd
import os
from scipy.linalg import lstsq # To increase efficiency, and double check the results
import sympy as sp
import re
np.set_printoptions(precision=5,linewidth=100) # adjust setting for prettier printing
import tkinter as tk
import tkinter.filedialog
%gui tk
# create a gui to let user select the working folder
# root=tk.Tk()
# working_folder = tkinter.filedialog.askdirectory(parent=root,title='Please select a directory')
# root.destroy() # close the gui after user selects the input 
# print(working_folder)

In [2]:
working_folder='E:/Ender/coursework/photogrammetry/Ass2'

In [3]:
#extract files from the working folder
file_list=os.listdir(working_folder)
for i in file_list:
    if i.endswith('.icf'):
        icf=i # extract icf filename
    elif i.endswith('.prj'):
        prj=i # extract prj filename
    elif i.endswith('.ori'):
        ori=i # extract ori filename
    elif i.endswith('.cam'):
        cam=i # extract cam filename
    elif i.endswith('.gcp'):
        gcp=i

In [4]:
os.chdir(working_folder) # change the path to working_folder
# Read .prj file to get the max iteration times and threshold for stopping the iteration
with open(prj,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('>>|\ |\n',txt)) # split the text so that I can index the value.
max_iteration=np.int32(txt[-7]) # Max iteration times
maxSigma=np.float32(txt[-6]) # The maximum allowable difference between unit weight variances in successive iterations. If the difference is less than this value, then the iterations will stop.
print('max_iteration:', max_iteration)
print('maxSigma:', maxSigma)

# Read .icf to get xy (image coordinates)
xy_img=pd.read_csv(icf,sep='\t|\   |\  ',engine='python',names=['Image_ID','Point_ID','x','y','w1','w2','w3','w4'])
## Subset the xy for each image
xy_img_09=xy_img[xy_img['Image_ID']=='65_09_20180803.jpg'] # subset the dataframe to get xy for 65_09_20180803.jpg
xy_img_10=xy_img[xy_img['Image_ID']=='65_10_20180803.jpg'] # repeat the process for the other images.
xy_img_14=xy_img[xy_img['Image_ID']=='65_14_20180803.jpg']
xy_img_18=xy_img[xy_img['Image_ID']=='65_18_20180803.jpg']
# Read IOP based on the output from lab 1
with open(cam,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('\t|\n',txt)) # split the text so that I can index the value.
xp,yp,c=np.float32(txt[9:12]) # extract xp,yp,c

print(f'xp={xp}, yp={yp}, c={c}')

k1,k2,k3,p1,p2=np.float32(txt[22:27]) #extract distortion parameters
print(f'k1={k1}, k2={k2}, k3={k3}, p1={p1}, p2={p2}')
# Read GCP from .gcp
gcp=pd.read_csv(gcp,engine='python',sep='\t|\   ',names=['Point_ID','X','Y','Z'],usecols=[0,1,2,3])
X_gcp=np.array(gcp['X']) # extract x of gcp
Y_gcp=np.array(gcp['Y'])
Z_gcp=np.array(gcp['Z'])
# Read .ori as initial guess for EOP for image9,10,14,18
with open(ori,'r') as f:
    line=0
    for i in f:
        line=line+1 # count line number
        text=f.readline()
        if line==9: # image 9
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_09,phi_09,kpp_09,Xo_09,Yo_09,Zo_09=(text[1:7])#extract EOP (initial guess)
        if line == 10:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_10,phi_10,kpp_10,Xo_10,Yo_10,Zo_10=(text[1:7])#extract EOP (initial guess)
        if line == 14:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_14,phi_14,kpp_14,Xo_14,Yo_14,Zo_14=(text[1:7])#extract EOP (initial guess)
        if line ==18:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_18,phi_18,kpp_18,Xo_18,Yo_18,Zo_18=(text[1:7])#extract EOP (initial guess)

max_iteration: 200
maxSigma: 1e-12
xp=0.06722940504550934, yp=-0.11779332906007767, c=8.15868854522705
k1=-0.0002753081498667598, k2=8.035393875616137e-06, k3=-2.082014987081493e-07, p1=6.32564042462036e-05, p2=-7.169081800384447e-05


In [115]:
# angle conversion
def deg2rad(angle): return np.deg2rad(np.float32(angle)) # When I read the angles from the text files, they are string, so I need to convert the type first.
def rad2deg(angle): return np.rad2deg(angle) # convert radian to degree
def preprocessing(xy_img):
    '''
    This function is used to pre-process the data for LSA
    '''
    num_observation=len(xy_img['x']) #compute number of observation
    xx=np.array(xy_img['x']).astype(np.float32) # define x_img
    yy=np.array(xy_img['y']).astype(np.float32) # define y_img
    point_ID=np.array(xy_img['Point_ID']).astype(np.int16)
    # The following lines compute the distortion
    x_bar=xx-xp
    y_bar=yy-yp
    rd=np.sqrt(x_bar**2+y_bar**2) # distance from principle point
    dist_x=x_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p1*(rd**2+2*x_bar**2)+2*p2*x_bar*y_bar
    dist_y=y_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p2*(rd**2+2*y_bar**2)+2*p1*x_bar*y_bar
    # the following lines compute x_corrected and y_corrected
    x_corrected=xx-dist_x
    y_corrected=yy-dist_y
    return x_corrected,y_corrected, num_observation,point_ID
def get_matrix(x_corrected,y_corrected,xp,yp,num_observation,point_ID,X_gcp,Y_gcp):
    '''
    This function defines the f and A in f=AX for LSA
    '''
    x_temp=x_corrected-xp # define x in f
    y_temp=y_corrected-yp # define y in f
    f_ls=np.zeros((2*num_observation,1))
    A_ls=np.zeros((2*num_observation,8))
    for i in range(num_observation):
        n_gcp=point_ID[i]-1 # find the corresponding gcp ID
        # define f
        f_ls[2*i]=x_temp[i] 
        f_ls[2*i+1]=y_temp[i]
        # define A
        A_ls[2*i]=np.array([X_gcp[n_gcp],Y_gcp[n_gcp],1,0,0,0,-x_temp[i]*X_gcp[n_gcp],-x_temp[i]*Y_gcp[n_gcp]])
        A_ls[2*i+1]=np.array([0,0,0,X_gcp[n_gcp],Y_gcp[n_gcp],1,-y_temp[i]*X_gcp[n_gcp],-y_temp[i]*Y_gcp[n_gcp]])
    return f_ls,A_ls,x_temp,y_temp

In [116]:
# LSA: y=AX. y = [x,y].T X=[C1,C2,C3,C4...,C8].T
def LSA(xy_img_ID,X_gcp,Y_gcp,xp,yp):
    x_corrected,y_corrected, num_observation,point_ID=preprocessing(xy_img_ID) # call the preprocessing function in order to compute f and A in next line
    f_ls,A_ls,x_temp,y_temp=get_matrix(x_corrected,y_corrected,xp,yp,num_observation,point_ID,X_gcp,Y_gcp)# compute f and A and elements in f
    X_ls,sigma_new,rank,s=lstsq(A_ls,f_ls) # Compute X in f=AX. This function is more efficient
    # X_ls=np.linalg.inv(A_ls.T@A_ls)@A_ls.T@f # This line is equivalent to the previous line
    return X_ls,x_temp,y_temp,num_observation,point_ID


In [114]:
def get_EOP(X_gcp,Y_gcp,xy_img_ID):
    '''
    This function is the main function. It calls LSA and use the X from LSA to compute EOP
    '''
    #Call LSA function to get required parameters
    X_ls,x_temp,y_temp,num_observation,point_ID=LSA(xy_img_ID,X_gcp,Y_gcp,xp,yp) 
    C1,C2,C3,C4,C5,C6,C7,C8=X_ls.reshape(-1) # define elements in C matrix
    C_array=np.array([[C1,C2,C3],[C4,C5,C6],[C7,C8,1]]) # define C matrix
    c_new=np.sqrt(-(C1*C2+C4*C5)/(C7*C8)) # compute new c
    A=np.array([[1,0,0],[0,1,0],[0,0,-c_new]])@C_array # Compute A matrix (known)
    ATA=A.T@A #A.T@A=B.T@B B.T@B contains X0,Y0,Z0
    X0=-ATA[0][2]/ATA[0][0] # compute X0
    Y0=-ATA[1][2]/ATA[0][0] # compute Y0
    Z0=np.sqrt((ATA[2][2]/ATA[0][0])-X0**2-Y0**2) # compute Z0
    S=np.zeros((4,4))
    X_gcp_sub=[] # create a list for X_gcp in case the PointID is not 1-25
    Y_gcp_sub=[] # create a list for Y_gcp in case the PointID is not 1-25
    Z_gcp_sub=[] # create a list for Z_gcp in case the PointID is not 1-25
    for i in range(num_observation):
        n_gcp=point_ID[i]-1 # define the GCP ID
        # define xi array
        xi=np.array([x_temp[i],y_temp[i],-c_new]) 
        xi=xi/np.sqrt(xi@xi.T)
        # define Xi array 
        Xi=np.array([X_gcp[n_gcp]-X0,Y_gcp[n_gcp]-Y0,0-Z0])
        Xi=Xi/np.sqrt(Xi@Xi.T)
        # the following lines compute S matrix. 
        S1_temp=np.array([[0,-xi[0],-xi[1],-xi[2]],
                         [xi[0],0,xi[2],-xi[1]],
                         [xi[1],-xi[2],0,xi[0]],
                         [xi[2],xi[1],-xi[0],0]]) # component 1 for computing S
        S2_temp=np.array([[0,-Xi[0],-Xi[1],-Xi[2]],
                         [Xi[0],0,-Xi[2],Xi[1]],
                         [Xi[1],Xi[2],0,-Xi[0]],
                         [Xi[2],-Xi[1],Xi[0],0]]) # component 2 for computing S
        # compute S
        S_temp=S1_temp.T@S2_temp 
        S=S+S_temp
        X_gcp_sub.append(X_gcp[n_gcp]) # add the x coordinate of gcp to X_gcp_sub
        Y_gcp_sub.append(Y_gcp[n_gcp]) # add the y coordinate of gcp to Y_gcp_sub
        Z_gcp_sub.append(Z_gcp[n_gcp]) # add the z coordinate of gcp to Z_gcp_sub
    # convert the data type to numpy array
    X_gcp_sub=np.array(X_gcp_sub) 
    Y_gcp_sub=np.array(Y_gcp_sub)
    Z_gcp_sub=np.array(Z_gcp_sub)
    # Compute eigen value and eigen vector of S
    eig_value,eig_vector=np.linalg.eig(S)
    index=np.where(eig_value==np.max(eig_value))
    qo,qx,qy,qz=eig_vector[:,index].reshape(-1) # define quaternion
    # Use quaternion to define rotation matrix R
    R=np.array([[qo**2+qx**2-qy**2-qz**2,2*qx*qy-2*qo*qz,2*qx*qz+2*qo*qy],
                [2*qx*qy+2*qo*qz, qy**2-qz**2+qo**2-qx**2, 2*qy*qz-2*qo*qx],
                [2*qx*qz-2*qo*qy,2*qy*qz+2*qo*qx, qz**2-qy**2-qx**2+qo**2]])
    # compute omega, phi, kappa from R
    omega=np.arctan2(-R[1,2],R[2,2])
    phi=np.arcsin(R[0,2])
    kappa=np.arctan2(-R[0,1],R[0,0])
    # convert radian to degree
    omega=rad2deg(omega) 
    phi=rad2deg(phi)
    kappa=rad2deg(kappa)
    # The following lines compute the Nx,Ny,D in ordert to compute residue
    Nx=R[0,0]*(X_gcp_sub-X0)+R[1,0]*(Y_gcp_sub-Y0)+R[2,0]*(Z_gcp_sub-Z0)
    Ny=R[0,1]*(X_gcp_sub-X0)+R[1,1]*(Y_gcp_sub-Y0)+R[2,1]*(Z_gcp_sub-Z0)
    D =R[0,2]*(X_gcp_sub-X0)+R[1,2]*(Y_gcp_sub-Y0)+R[2,2]*(Z_gcp_sub-Z0)
    # compute residue in x and y directions
    res_x = x_temp + c_new*Nx/D
    res_y = y_temp + c_new*Ny/D
    # compute RMSE
    RMSE=np.sqrt(np.sum(res_x**2+res_y**2)/num_observation)

    print(RMSE)
get_EOP(X_gcp,Y_gcp,xy_img_09)

0.03531282906189339
